In [142]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('smallcap.csv')  # Make sure to replace '/path/to/your/smallcap.csv' with the actual file path

# Extract the tickers into a list
tickers = df['Symbol'].tolist()

# Now, 'tickers' contains all the tickers from the "Symbol" column of your CSV file
print(tickers)


['BMR', 'GKIN', 'OMH', 'MRAI', 'FLES', 'DKL', 'BNZI', 'GROM', 'OCTO', 'QTI', 'UK', 'HSCS', 'NBCO', 'MCOM', 'AGRI', 'BREA', 'HOLO', 'NXU', 'VLCN', 'HQGE', 'REBN', 'LTRN', 'SPRC', 'LYT', 'OCEA', 'RVSN', 'SMX', 'ASTI', 'AULT', 'APM', 'NKGN', 'CSTF', 'CERO', 'IVP', 'MIRA', 'SIRC', 'AIMD', 'SGD', 'PBM']


In [146]:
import requests
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
from datetime import datetime, timedelta


# Replace 'YOUR_API_KEY' with your actual Alpha Vantage API key
api_key = 'HGLDEL3XDK78QXPN'
symbol = 'IBM'  # Example symbol
interval = '5min'  # Data interval
outputsize = 'full'  # Get the full intraday data for a specific month
ts = TimeSeries(key=api_key, output_format='pandas')

tickers = [ 'GKIN', 'OMH', 'MRAI', 'FLES', 'DKL', 'BNZI', 'GROM', 'OCTO', 'QTI', 'UK', 'HSCS', 'NBCO', 'MCOM', 'AGRI', 'BREA', 'HOLO', 'NXU', 'VLCN', 'HQGE', 'REBN', 'LTRN', 'SPRC', 'LYT', 'OCEA', 'RVSN', 'SMX', 'ASTI', 'AULT', 'APM', 'NKGN', 'CSTF', 'CERO', 'IVP', 'MIRA', 'SIRC', 'AIMD', 'SGD', 'PBM']

# Function to generate the past 12 months in 'YYYY-MM' format
def generate_past_12_months():
    return [(datetime.now() - timedelta(days=30*i)).strftime('%Y-%m') for i in range(0, 13)]

# Generate the list of the past 12 months
months = generate_past_12_months()

success_list = []

for ticker in tickers:

    # Initialize a DataFrame to store all the data
    all_data_df = pd.DataFrame()
    results = []
    symbol=ticker

    print(f"Downloading daily data for {ticker} for the full history")
    try:
        # Get daily data
        daily_data, _ = ts.get_daily(symbol=ticker, outputsize='full')
        daily_data.sort_index(ascending=True, inplace=True)
        daily_data.index = pd.to_datetime(daily_data.index)
    except Exception as e:
        print(f"Error for {ticker}: {e}")

    print(f"Downloading intraday data for {ticker} for the past year")

    # Calculate the slices for the past 12 months from the current date
    for month in months[::-1]:
        try:
            url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&month={month}&outputsize={outputsize}&apikey={api_key}'
            response = requests.get(url)
            data = response.json()  # Data is in JSON format

            # The exact key for the time series data depends on the interval and function used
            # Adjust the key based on the actual structure of your JSON response
            time_series_key = f"Time Series ({interval})"

            if time_series_key in data:
                time_series_data = data["Time Series (5min)"]
                # Convert to DataFrame
                df = pd.DataFrame.from_dict(time_series_data, orient='index')
                df.index = pd.to_datetime(df.index)  # Converting index to datetime
                df.sort_index(inplace=True)  # Sorting by datetime index

                # Concatenate with the main DataFrame
                all_data_df = pd.concat([all_data_df, df])

            else:
                print(f"No data found for month: {month}")
                break

            for day in pd.unique(df.index.date):
                day_str = day.strftime('%Y-%m-%d')
                day_data = all_data_df.between_time('09:30', '10:30').loc[day_str]
                if not day_data.empty:
                    open_price  = float(day_data.iloc[0]['1. open'])
                    close_price = float(day_data.iloc[-1]['4. close'])
                    high_price  = float(day_data['2. high'].max())
                    low_price   = float(day_data['3. low'].min())

                    # Find the last trading day's close price
                    today_open = daily_data.loc[day_str,'1. open']
                    previous_days_data = daily_data[daily_data.index < day_str]

                    # Calculate the gap percentage compared to yesterday's close
                    if not previous_days_data.empty:
                        previous_day_close = previous_days_data.iloc[-1]['4. close']
                        previous_day_open = previous_days_data.iloc[-1]['1. open']
                        previous_day_high = previous_days_data.iloc[-1]['2. high']
                        previous_day_low  = previous_days_data.iloc[-1]['3. low']
                        previous_day_volume  = previous_days_data.iloc[-1]['5. volume']

                        two_days_ago_close = previous_days_data.iloc[-2]['4. close']
                        two_days_ago_open = previous_days_data.iloc[-2]['1. open']
                        two_days_ago_high = previous_days_data.iloc[-2]['2. high']
                        two_days_ago_low  = previous_days_data.iloc[-2]['3. low']
                        two_days_ago_volume = previous_days_data.iloc[-2]['5. volume']

                        # print(type(previous_day_close), type(open_price))
                        gap_percentage = ((today_open - previous_day_close) / previous_day_close) * 100
                    else:
                        gap_percentage = None

                    results.append([day, ticker, open_price, close_price, high_price, low_price, gap_percentage, previous_day_close, previous_day_open, previous_day_high, previous_day_low, previous_day_volume, two_days_ago_close, two_days_ago_open, two_days_ago_high, two_days_ago_low, two_days_ago_volume])

        except Exception as e:
            print(f"Error for {ticker}: {e}")

    # # # Convert the results into a DataFrame with an additional column for the gap percentage
    results_df = pd.DataFrame(results, columns=['Date', 'Ticker', '9:30 Price', '10:30 Price', 'High Price', 'Low Price', 'Gap Percentage', 'previous_day_close',  'previous_day_open', 'previous_day_high', 'previous_day_low', 'previous_day_volume', 'two_days_ago_close', 'two_days_ago_open', 'two_days_ago_high', 'two_days_ago_low', 'two_days_ago_volume'])

    results_df.to_csv(f'{ticker}.csv', index=False)
    if not results_df.empty:
        success_list.append(ticker)
        print(f'---> Data retrival successful for {ticker}')

No data found for month: 2023-03
Error for OMH: '2023-03-21'
data retrival successful for OMH
data retrival successful for MRAI
No data found for month: 2023-03


KeyboardInterrupt: 

In [141]:
# print(results_df.head())
# # results_df.to_csv(f'{ticker}.csv', index=False)

print(daily_data)
# # daily_data, _ = ts.get_daily(symbol=ticker, outputsize='full')
# # daily_data.sort_index(ascending=True, inplace=True)
# # daily_data.index = pd.to_datetime(daily_data.index)
# # daily_data.to_csv('daily_data.csv', index=True)

# print(months)
# # previous_days_data = daily_data[daily_data.index < '2023-03-02']
# print(previous_days_data)
# print(daily_data.head())

# previous_day_close = previous_days_data.iloc[-1]['4. close']
# open_price  = float(daily_data.loc['2023-03-02', '1. open'])
# print(previous_day_close,open_price)

# for day in pd.unique(all_data_df.index.date):
#     day_str = day.strftime('%Y-%m-%d')

print(df)
print(all_data_df)
print(results_df)

            1. open  2. high   3. low  4. close  5. volume
date                                                      
1999-11-01    1.125    1.125   1.1250    1.1250        0.0
1999-11-02    1.125    1.125   1.1250    1.1250        0.0
1999-11-03    1.125    1.125   1.1250    1.1250        0.0
1999-11-04    1.125    1.125   1.0625    1.0625        0.0
1999-11-05    1.125    1.125   1.0625    1.0625      100.0
...             ...      ...      ...       ...        ...
2024-03-04   34.500   34.500  32.0200   32.0200      604.0
2024-03-05   32.020   32.020  32.0200   32.0200        0.0
2024-03-06   32.020   32.020  32.0200   32.0200        0.0
2024-03-07   32.020   32.020  32.0200   32.0200        0.0
2024-03-08   32.020   32.020  32.0200   32.0200        0.0

[6108 rows x 5 columns]
                     1. open  2. high   3. low 4. close 5. volume
2023-05-01 09:30:00  10.6400  11.1100  10.6100  10.9960      3700
2023-05-01 13:20:00  10.6800  10.7200  10.6800  10.7200      5295
2023-05-01

In [126]:
print(df)

                      1. open   2. high    3. low  4. close 5. volume
2024-03-01 04:15:00  185.4600  185.4600  185.4600  185.4600         2
2024-03-01 04:25:00  185.4700  185.4700  185.4600  185.4600         2
2024-03-01 04:30:00  185.0600  185.4600  185.0600  185.4600         2
2024-03-01 04:35:00  185.4500  185.4500  185.4500  185.4500         1
2024-03-01 04:45:00  185.0900  185.0900  185.0900  185.0900        71
...                       ...       ...       ...       ...       ...
2024-03-08 19:35:00  196.2300  196.2300  196.2300  196.2300         2
2024-03-08 19:40:00  195.4900  195.9400  195.4900  195.9400         8
2024-03-08 19:45:00  195.9500  195.9500  195.4900  195.5000        32
2024-03-08 19:50:00  195.5000  195.5000  195.5000  195.5000         3
2024-03-08 19:55:00  195.4900  195.4900  195.4800  195.4800       105

[1072 rows x 5 columns]
